<a href="https://colab.research.google.com/github/nathrod/TT003A/blob/main/Trabalho_1_TT003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wine Quality

Objetivo: mostrar que consegue utilizar redes neurais em um cenário de aprendizado supervisionado básico.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df = pd.read_csv(url, delimiter=';')

Conjunto de dados de análise de vinhos tintos, que contém informações químicas e sensoriais de 1.599 amostras. Ele possui 12 colunas, sendo 11 variáveis químicas e uma variável de qualidade sensorial, avaliada em uma escala de 3 á 8.

In [ ]:
# Apresentação dos dados
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
df.describe(include="all")

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


 A avaliação de qualidade tem uma média de aproximadamente 5,6, com valores mínimos de 3 e máximos de 8, indicando uma maioria de vinhos com qualidade intermediária.

In [ ]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [ ]:
df.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


Este dataset de análise de vinhos tintos não possui valores nulos, o que facilita o trabalho com as variáveis sem a necessidade de tratar dados ausentes. Nosso objetivo é prever a qualidade do vinho com base nas demais características químicas, sendo a coluna de qualidade (quality) nosso target. Para realizar essa previsão, iremos dividir o dataset em features (X) e rótulo (y): as features contêm todas as variáveis independentes que descrevem as propriedades químicas dos vinhos, enquanto o rótulo representa a variável de qualidade a ser prevista.

### Esse vinho é de alta qualidade ou não?

Classificação Binária. Vantagens:

*   Menor complexidade do problema ao dividir em alta e baixa qualidade;
*   Uma classificação binária é mais fácil de interpretar, especialmente para não especialistas, auxiliando na tomada de decisões de produção e marketing.
*   Foco em um Objetivo Específico

Com essa categorização, podemos desenvolver um modelo mais objetivo, capaz de responder diretamente à questão.

Optamos por usar uma classificação binária para simplificar o modelo e torná-lo mais intuitivo, categorizando a qualidade do vinho em alta ou baixa. Ao invés de prever uma nota específica, decidimos por um limite: vinhos com qualidade 7 ou mais são classificados como 1 (alta qualidade), e os demais como 0 (baixa qualidade).  


A classificação binária é uma escolha prática quando o objetivo é diferenciar produtos de qualidade superior daqueles de qualidade média ou inferior, sem entrar em detalhes das notas específicas. Isso é útil, pois muitas vezes a decisão final para o consumidor ou para o produtor é saber se o vinho é "bom" ou "regular" ao invés de uma classificação numérica exata.

In [ ]:
# Dividindo features e rótulo
X = df.drop(columns=['quality'])
y = df['quality']

# Transformando a tarefa em classificação binária
y = (y >= 7).astype(int)

# Normalizando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

Criamos uma rede neural simples chamada WineNet usando o PyTorch para classificar a qualidade do vinho. A rede neural herda de nn.Module e possui três camadas totalmente conectadas: a primeira com 64 neurônios, uma camada intermediária com 32, e uma camada de saída com um único neurônio para classificação binária. Um dropout de 30% é aplicado após a primeira camada para evitar overfitting. A função forward define a passagem dos dados pela rede, aplicando a função ReLU nas camadas intermediárias.

In [ ]:
# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo para tensores do PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

# 3 - Criando uma rede neural usando o método de herança de nn.Module.
class WineNet(nn.Module):
    def __init__(self):
        super(WineNet, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

**Função de custo utilizada:** Binary Cross Entropy Loss (nn.BCEWithLogitsLoss)    
Escolhemos essa função por conta do problema ser uma classificação binária. Essa função é ideal, pois combina a sigmoid e o cálculo de binary cross-entropy em uma única etapa, tornando o treinamento mais eficiente e estável numericamente.

**Otimização:** Adam (torch.optim.Adam), eficiência e ajuste dinâmico

In [ ]:
# 4/5. Instanciando o modelo e definindo a função de custo e otimizador
model = WineNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Treinamento da rede neural utilizando mini-batches e backpropagation. Em cada época, criamos uma ordem aleatória dos dados para maior variabilidade e, em seguida, dividimos o conjunto de treino em lotes de 32 amostras cada. A cada 10 épocas, exibimos o valor da perda para acompanhar o progresso do treinamento.

In [ ]:
# 6. Treinamento com Mini-Batches
epochs = 100
batch_size = 32
n_batches = X_train.size(0) // batch_size

for epoch in range(epochs):
    permutation = torch.randperm(X_train.size(0))

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        indices = permutation[start:end]
        batch_x, batch_y = X_train[indices], y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 10/100, Loss: 0.3503
Epoch 20/100, Loss: 0.1594
Epoch 30/100, Loss: 0.2166
Epoch 40/100, Loss: 0.1335
Epoch 50/100, Loss: 0.1565
Epoch 60/100, Loss: 0.1314
Epoch 70/100, Loss: 0.1648
Epoch 80/100, Loss: 0.2216
Epoch 90/100, Loss: 0.1208
Epoch 100/100, Loss: 0.1443


Avaliando o modelo passamos o x_test.

In [ ]:
# 7. Avaliação no conjunto de teste
with torch.no_grad():
    outputs = model(X_test).squeeze()
    predicted = torch.round(torch.sigmoid(outputs))
    accuracy = accuracy_score(y_test, predicted)
    print(f'Acurácia no conjunto de teste: {accuracy:.4f}')
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, predicted))
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, predicted))

Acurácia no conjunto de teste: 0.9000

Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94       273
         1.0       0.69      0.57      0.63        47

    accuracy                           0.90       320
   macro avg       0.81      0.77      0.79       320
weighted avg       0.89      0.90      0.90       320


Matriz de Confusão:
[[261  12]
 [ 20  27]]


##### Conclusões:

*   Acurácia 90% - 90% dos dados são classificados corretamente.
Analisando a matriz de confusão temos:

* O modelo corretamente classificou 261 vinhos como baixa qualidade e 27 vinhos como alta qualidade.
* Houve 12 falsos positivos (previu alta qualidade quando era baixa) e 20 falsos negativos (previu baixa qualidade quando era alta).
